In [1]:
from dotenv import load_dotenv
import sys

sys.path.append("../common")
load_dotenv()

True

In [6]:
import os
from langsmith_tracker import set_tracking

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
set_tracking(project_name="Chains-SQL")

Langsmith 추적이 활성화되었습니다. [프로젝트명: Chains-SQL]


## SQL

`create_sql_query_chain` 을 활용하여 생성한 체인으로 SQL 쿼리를 생성 할 수 있습니다.  
  
SQL 쿼리를 생성한 후 실행하고 답변을 도출하는 방법입니다.

### 1. SQL Database 정보를 불러옵니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

# 데이터 베이스 연결
db = SQLDatabase.from_uri("sqlite:///data/finance.db")

# 데이터베이스 출력
print(db.dialect)

# 사용 가능한 테이블 이름들 출력
print(db.get_usable_table_names())

### 2. LLM 과 DB 를 매개변수로 입력하여 chain 을 생성합니다.

In [ ]:
# model 은 gpt-3.5-turbo 를 지정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# LLM 과 DB 를 매개변수로 입력하여 chain 을 생성합니다.
chain = create_sql_query_chain(llm, db)

### 3. 프롬프트를 작성합니다.

내용으로 `table_info`, `column_description` 을 포함하여 작성할 수 있습니다.

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:
{table_info}

Here is the description of the columns in the tables:
`cust`: customer name
`prod`: product name
`trans`: transaction date

Question: {input}"""
).partial(dialect=db.dialect)

# model 은 gpt-3.5-turbo 를 지정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# LLM 과 DB 를 매개변수로 입력하여 chain 을 생성합니다.
chain = create_sql_query_chain(llm, db, prompt)

### 4. chain 을 실행하면 DB 기반으로 쿼리를 생성합니다.

In [ ]:
# chain 을 실행하고 결과를 출력합니다.
generated_sql_query = chain.invoke({"question": "고객의 이름을 나열하세요"})

# 생성된 쿼리를 출력합니다.
print(generated_sql_query.__repr__())

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

# 생성한 쿼리를 실행하기 위한 도구를 생성합니다.
execute_query = QuerySQLDataBaseTool(db=db)

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

# 도구
execute_query = QuerySQLDataBaseTool(db=db)

# SQL 쿼리 생성 체인
write_query = create_sql_query_chain(llm, db)

# 생성한 쿼리를 실행하기 위한 체인을 생성합니다.
chain = write_query | execute_query

In [ ]:
# 실행 결과 확인
chain.invoke({"question": "***을 조회하세요"})

### 6. 답변을 LLM 으로 증강-생성

이전 단계에서 생성한 chain 을 사용하면 답변이 단답형 형식으로 출력됩니다. 이를 LCEL 문법의 체인으로 좀 더 자연스러운 답변을 받을 수 있도록 조정할 수 있습니다.

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()

# 생성한 쿼리를 실행하고 결과를 출력하기 위한 체인을 생성합니다.
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

In [ ]:
# 실행 결과 확인
chain.invoke({"question": "*** 를 조회하세요"})

## Agent

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

# model 은 gpt-3.5-turbo 를 지정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# SQLite 데이터베이스에 연결합니다.
db = SQLDatabase.from_uri("sqlite:///data/finance.db")

# Agent 생성
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [ ]:
# 실행 결과 확인
agent_executor.invoke(
    {"input": "*** 를 조회하세요"}
)